<a href="https://colab.research.google.com/github/marina554/Consolidated-Accounting/blob/main/Handles_both_B_S_and_P_L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================================
# 連結会計シンプルモデル（B/S＋P/L対応版）
# =========================================
import pandas as pd

# ---------------------------
# 1. 各社の単体財務データ
# ---------------------------

# 親会社
parent_bs = {
    "現金": 1000,
    "売掛金": 500,
    "棚卸資産": 300,
    "子会社株式": 300,  # 投資勘定（今回は簡略化）
    "買掛金": 400,
    "親子間売掛金": 200,  # 内部取引用
    "資本金": 1000,
    "利益剰余金": 700
}

parent_pl = {
    "売上高": 2000,
    "売上原価": 800,
    "販売費及び一般管理費": 300,
    "親子間売上高": 200,  # 子会社向けの内部売上
}

# 子会社
child_bs = {
    "現金": 300,
    "売掛金": 100,
    "棚卸資産": 100,
    "親子間買掛金": 200,  # 親会社への内部取引
    "買掛金": 150,
    "資本金": 200,
    "利益剰余金": 150
}

child_pl = {
    "売上高": 500,
    "売上原価": 200,
    "販売費及び一般管理費": 100,
    "親子間仕入": 200  # 親会社からの内部仕入
}

# 持分比率（親会社が子会社株式を保有する割合）
ownership = 0.6  # 60%

# ---------------------------
# 2. 内部取引消去関数（B/S & P/L）
# ---------------------------
def eliminate_internal_transactions(parent_pl, child_pl, parent_bs, child_bs):
    """
    親子間の内部売上・仕入、および債権債務を消去
    """
    # 内部取引金額を同額と仮定
    internal_amount = min(parent_pl["親子間売上高"], child_pl["親子間仕入"])

    # 売上・仕入の消去
    parent_pl["親子間売上高"] -= internal_amount
    child_pl["親子間仕入"] -= internal_amount

    # 債権債務の消去
    internal_bs_amount = min(parent_bs["親子間売掛金"], child_bs["親子間買掛金"])
    parent_bs["親子間売掛金"] -= internal_bs_amount
    child_bs["親子間買掛金"] -= internal_bs_amount

    print(f"内部取引消去（売上・仕入）: {internal_amount}")
    print(f"内部取引消去（債権・債務）: {internal_bs_amount}")

# 実行
eliminate_internal_transactions(parent_pl, child_pl, parent_bs, child_bs)

# ---------------------------
# 3. 連結B/S作成
# ---------------------------
def create_consolidated_bs(parent, child, ownership):
    consolidated = {}
    for key in set(parent.keys()) | set(child.keys()):
        consolidated[key] = parent.get(key, 0) + child.get(key, 0)

    # 非支配株主持分（子会社の純資産×持分非保有率）
    child_net_assets = child["資本金"] + child["利益剰余金"]
    nci_bs = child_net_assets * (1 - ownership)
    consolidated["非支配株主持分"] = nci_bs
    return consolidated

# ---------------------------
# 4. 連結P/L作成
# ---------------------------
def create_consolidated_pl(parent_pl, child_pl, ownership):
    consolidated = {}
    for key in set(parent_pl.keys()) | set(child_pl.keys()):
        consolidated[key] = parent_pl.get(key, 0) + child_pl.get(key, 0)

    # 内部取引項目は既に消去済なので無視
    consolidated.pop("親子間売上高", None)
    consolidated.pop("親子間仕入", None)

    # 当期純利益（超シンプル計算）
    consolidated["営業利益"] = (
        consolidated["売上高"]
        - consolidated["売上原価"]
        - consolidated["販売費及び一般管理費"]
    )

    # 子会社当期純利益（非支配株主持分のため）
    child_net_income = child_pl["売上高"] - child_pl["売上原価"] - child_pl["販売費及び一般管理費"]
    nci_pl = child_net_income * (1 - ownership)
    consolidated["非支配株主持分当期純利益"] = nci_pl
    consolidated["親会社株主に帰属する当期純利益"] = consolidated["営業利益"] - nci_pl

    return consolidated

# ---------------------------
# 5. 実行・結果表示
# ---------------------------
consolidated_bs = create_consolidated_bs(parent_bs, child_bs, ownership)
consolidated_pl = create_consolidated_pl(parent_pl, child_pl, ownership)

df_bs = pd.DataFrame([consolidated_bs], index=["連結B/S"])
df_pl = pd.DataFrame([consolidated_pl], index=["連結P/L"])

print("\n==== 連結貸借対照表（B/S） ====")
print(df_bs.T)

print("\n==== 連結損益計算書（P/L） ====")
print(df_pl.T)
